In [98]:
# Importing the pandas library for data manipulation
import pandas as pd 

# Importing the re library for regular expression operations
import re 

# Importing the os library for operating system dependent functionality
import os 

# Importing the mysql.connector library for connecting to MySQL databases
import mysql.connector as msql 

# Importing the math library for mathematical operations
import math

import numpy as np

In [99]:
# Specifying the directory path where the Excel files are located
directory_path = r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\Accelerator 3.0\data_files\General Information"

# Creating a list of Excel files in the specified directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Looping through each Excel file
for file in excel_files:
    # Constructing the full file path
    file_path = os.path.join(directory_path, file)
    
    # Reading the Excel file into a DataFrame
    gi = pd.read_excel(file_path)
    
    # Printing the name of the file
    print(f"Data from {file}:")

Data from General Information.xlsx:


In [100]:
# Displaying the first few rows of the DataFrame
gi.head()

,Mentee ID,Email ID,Mentee Name,Mobile Number,Group Name
0,KA03_01,rangarirohinireddy@gmail.com,Rohini Reddy Rangari,6.281661e+09,BIO_01Neha
1,KA03_02,namrtad82@gmail.com,Namrta Dixit,9.630908e+09,BIO_01Neha
2,KA03_03,Amalendus4nair@gmail.com,Amalendu Sasikumar Nair,9.496757e+09,BIO_01Neha
3,KA03_26,anaswara2928@gmail.com,Anaswara H,7.736603e+09,BIO_01Neha
4,KA03_05,swathi.deepak@mbty.christuniversity.in,Swathi Nayak,9.606190e+09,BIO_01Neha


In [101]:
# Function to clean email addresses
def clean_email(email):
    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
gi['Email ID'] = gi['Email ID'].apply(clean_email)

In [102]:
# To see shape of dataframe
gi.shape

(136, 5)

In [103]:
# URL of the Google Sheets document

url = 'https://docs.google.com/spreadsheets/d/1jHRegtx-z0BtvWVPBi7p6kAouPmYDGB52F3CMCAjJ3I/edit?usp=sharing'

# The URL needs to be modified to point to the export format

url = url.replace('/edit?usp=sharing', '/export?format=xlsx')

# Read the data from the URL into a pandas DataFrame

at = pd.read_excel(url)
pt = pd.read_excel(url)

In [104]:
# To display all the columns
pd.set_option('display.max_columns', None)

In [105]:
# Renaming the 
at = at.rename(columns={'Select Week': 'Week',
                        'How many live mentor-mentee meetings did you conduct this week? ': 'Total Sessions',
                       'On average, how long were your  live Mentor - Mentee meetings? (In Minutes)': 'Average Duration'})


In [106]:
#at = at.rename(columns={'Select Week': 'Week'})

In [107]:
# Get the columns to keep
columns_to_keep = [col for col in at.columns if col.startswith("Week") or col.startswith("Attendance for this") or col.startswith("Total Sessions")or col.startswith("Average Duration")]

# Drop the columns not in columns_to_keep
at.drop(columns=[col for col in at.columns if col not in columns_to_keep], inplace=True)


In [108]:
# Extract the week number from the 'Select Week' column
at['Week'] = at['Week'].str.split(':').str[0]



In [109]:
# Remove comment before column names
at.columns = at.columns.str.replace("Attendance for this week's Mentor-Mentee interaction session", '')

In [110]:
# Remove extra spaces before column names
at.columns = at.columns.str.strip()

In [111]:
# Remove unwanted Sign from column names
at = at.rename(columns=lambda x: x.replace('[', '').replace(']', ''))


In [112]:
# Rename columns using a lambda function
at = at.rename(columns=lambda x: '_'.join(x.split('_')[:2]) if '_' in x else x)

In [113]:
# Unpivot the DataFrame 'at' by converting columns into rows
at_filter = pd.melt(at, id_vars=['Week','Total Sessions','Average Duration'], 
                    var_name='Mentee ID', value_name='Attended Sessions')

In [114]:
#pivoted_at_filter = at_filter.pivot_table(index='Mentee ID', columns='Week', values=['Total Sessions', 'Average Duration', 'Attended Sessions'])


In [115]:
# Checking the column names
at_filter.columns

Index(['Week', 'Total Sessions', 'Average Duration', 'Mentee ID',
       'Attended Sessions'],
      dtype='object')

In [116]:
# Getting all unique values to check
unique_attendance_values = at_filter['Attended Sessions'].unique()
print(unique_attendance_values)

[nan  0.  1.  2.]


In [117]:
# Replacing the values so we get data into int type
at_filter['Total Sessions'] = at_filter['Total Sessions'].replace({'More than 2': 2})
at_filter['Average Duration'] = at_filter['Average Duration'].replace({'More than 120': 120})



In [118]:
# Droping unwanted column
df_cleaned = at_filter.dropna(subset=['Attended Sessions'])


In [119]:
# Looking for duplicates
df_cleaned = df_cleaned.drop_duplicates()


In [120]:
# Create a pivot table for weekly data
at_pivot = df_cleaned.pivot_table(index='Mentee ID', columns='Week')


In [121]:
# Filling null values with 0
at_pivot.fillna(0, inplace=True)


In [122]:
# creating new column names by combining the first and second elements
at_pivot.columns = [f'{col[0]} - {col[1]}' for col in at_pivot.columns]


In [123]:
# Reseting the index
at_pivot.reset_index(inplace=True)

In [124]:
#at_pivot.tv.pl

In [125]:
def add_hours_spent_columns(df):
    # Extract week numbers from the columns
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    for week in weeks:
        attended_sessions_col = f'Attended Sessions - Week {week}'
        average_duration_col = f'Average Duration - Week {week}'
        hours_spent_col = f'Hours Spent - Week {week}'
        
        if attended_sessions_col in df.columns and average_duration_col in df.columns:
            df[hours_spent_col] = (df[attended_sessions_col] * df[average_duration_col]) / 60
    
    return df



In [126]:
def add_per_att_columns(df):
    # Extract week numbers from the columns
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    for week in weeks:
        total_sessions_col = f'Total Sessions - Week {week}'
        #attended_sessions_col = f'Attended Sessions - Week {week}'
        average_duration_col = f'Average Duration - Week {week}'
        hours_spent_col = f'Hours Spent - Week {week}'
        percentage_attendance_col = f'% Attendance - Week {week}'
        
        
        if hours_spent_col in df.columns and total_sessions_col in df.columns and average_duration_col in df.columns:
            df[percentage_attendance_col] = (df[hours_spent_col]/(df[total_sessions_col] * (df[average_duration_col]/60)))*100
    
    return df



def add_hours_spent_columns(df):
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    for week in weeks:
        attended_sessions_col = f'Attended Sessions - Week {week}'
        average_duration_col = f'Average Duration - Week {week}'
        hours_spent_col = f'Hours Spent - Week {week}'
        
        if attended_sessions_col in df.columns and average_duration_col in df.columns:
            df[hours_spent_col] = (df[attended_sessions_col] * df[average_duration_col]) / 60
    
    return df



In [127]:
# Useing above function we define
at_pivot = add_hours_spent_columns(at_pivot)
at_pivot = add_per_att_columns(at_pivot)

In [128]:
def rearrange_columns(df):
    # Extract week numbers from the columns
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    # Define the initial part of the arrangement
    arranged_columns = ['Mentee ID']
    
    # Arrange columns for each week dynamically
    for week in weeks:
        arranged_columns.append(f'Total Sessions - Week {week}')
        arranged_columns.append(f'Average Duration - Week {week}')
        arranged_columns.append(f'Attended Sessions - Week {week}')
        arranged_columns.append(f'Hours Spent - Week {week}')
        arranged_columns.append(f'% Attendance - Week {week}')
        
    return arranged_columns

In [129]:
# Loop through each column in the DataFrame
for column in at_pivot.columns:
    if column.startswith('% Attendance'):
        at_pivot[column] = at_pivot[column].apply(lambda x: 100 if x > 100 else x)

In [130]:
# Divide columns starting with 'Average Duration' by 60 to make it into Hours
for col in at_pivot.columns:
    if col.startswith('Average Duration'):
        at_pivot[col] = at_pivot[col] / 60

In [131]:
rearrange_columns = rearrange_columns(at_pivot)
# Rearrange the dataframe
at_pivot = at_pivot[rearrange_columns]


In [132]:
at_pivot.shape

(103, 26)

In [133]:
# Filling null values with 0
at_pivot.fillna(0, inplace=True)

In [134]:
# Creating function to get Overall Attendance columns
def calculate_mean_scores(df, prefixes):
# Calculate overall scores for the specified prefixes
    for prefix in prefixes:
        # Filter columns that start with the given prefi
        cols = [col for col in df.columns if col.startswith(prefix)]
        # Calculate the sum of scores across columns for the current prefix
        df[f"{prefix} Overall"] = df[cols].sum(axis=1)  # Calculate overall score
        
# Usage of Function
prefixes = ["Total Sessions", "Attended Sessions", "Hours Spent"]

calculate_mean_scores(at_pivot, prefixes)


### Below👇 , we haven't taken the average of those classes, which were not taken by the mentor. The average is of those classes only taken by the mentor, not by week-wise classes. If there are 14 weeks and the menotr has taken a class of 8 weeks, than we have taken an average of just 8 👇

In [135]:
# List of columns
attendance_cols = [col for col in at_pivot.columns if '% Attendance' in col]
session_cols = [col for col in at_pivot.columns if 'Total Sessions' in col]

# Function to calculate the overall % attendance
def calculate_overall_attendance(row):
    total_attendance = 0
    count = 0
    for att_col, sess_col in zip(attendance_cols, session_cols):
        if row[sess_col] != 0:
            total_attendance += row[att_col]
            count += 1
    return total_attendance / count if count > 0 else 0

# Apply the function to each row
at_pivot['% Attendance Overall'] = at_pivot.apply(calculate_overall_attendance, axis=1)




In [136]:
# Merge with gi
at_pivot = pd.merge(gi, at_pivot, on='Mentee ID', how='outer')


In [137]:
# Filling null vaules
at_pivot.fillna(0, inplace=True)


In [138]:
at_pivot=at_pivot.round(2)

In [139]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\Accelerator 3.0\output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Mentor Interaction Attendance.csv')
at_pivot.to_csv(output_path, index=False)


# Project Tracking

In [140]:
# Copy dataset for using it agian
pj = pt.copy()

In [141]:
# Get the columns to keep
columns_to_keep = [col for col in pj.columns if col.startswith("Select Week") or col.startswith("Attendance for this") or col.startswith("Select the 7E learning")or col.startswith("Enlist the tasks assigned") or col.startswith("Select the theme of the week.")]

# Drop the columns not in columns_to_keep
pj.drop(columns=[col for col in pj.columns if col not in columns_to_keep], inplace=True)


In [142]:
# Checking columns names
pj.columns

Index(['Select Week', 'Select the 7E learning phase for this week.',
       'Enlist the tasks assigned', 'Select the theme of the week.',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_01_Rohini Reddy Rangari]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_02_Namrta Dixit]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_03_Amalendu Sasikumar Nair]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_04_Anjali Pant]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_05_Swathi Nayak]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_26_Anaswara H]',
       ...
       'Attendance for this week's Mentor-Mentee interaction session [KA03_129_Anu M]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_130_Himani Upadhayay]',
       'Attendance for this week's Mentor-Mentee interaction session [KA03_131_Hema 

In [143]:
# Merging three columns into one and sepreating it by '|'
pj['Project Details'] = (
    pj['Select the 7E learning phase for this week.'] + ' | ' +
    pj['Select the theme of the week.'] + ' | ' +
    pj['Enlist the tasks assigned']
)

# Drop the original columns if necessary
pj.drop(['Select the 7E learning phase for this week.', 'Enlist the tasks assigned', 'Select the theme of the week.'], axis=1, inplace=True)


In [144]:
# Extract the week number from the 'Select Week' column
pj['Select Week'] = pj['Select Week'].str.split(':').str[0]



In [145]:
# Remove comment before column names
pj.columns = pj.columns.str.replace("Attendance for this week's Mentor-Mentee interaction session", '')

In [146]:
# Remove extra spaces before column names
pj.columns = pj.columns.str.strip()

In [147]:
# Remove unwanted Sign from column names
pj = pj.rename(columns=lambda x: x.replace('[', '').replace(']', ''))


In [148]:
# Rename columns using a lambda function
pj = pj.rename(columns=lambda x: '_'.join(x.split('_')[:2]) if '_' in x else x)

In [149]:
# Unpivot the DataFrame 'pj' by converting columns into rows
pj_filter = pd.melt(pj, id_vars=['Select Week','Project Details'], 
                    var_name='Mentee ID', value_name='Attended Sessions')

In [150]:
# Checking head of dataframe
pj_filter.head()

,Select Week,Project Details,Mentee ID,Attended Sessions
0,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
1,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
2,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
3,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
4,Week 2,Elicit - Week 1-2 | Week 2 : Project Planning ...,KA03_01,NaN


In [151]:
# Droping unwanted column
pj_cleaned = pj_filter.dropna(subset=['Attended Sessions'])


In [152]:
pj_filter

,Select Week,Project Details,Mentee ID,Attended Sessions
0,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
1,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
2,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
3,Week 1,Elicit - Week 1-2 | Week 1 : Project Initiatio...,KA03_01,NaN
4,Week 2,Elicit - Week 1-2 | Week 2 : Project Planning ...,KA03_01,NaN
...,...,...,...,...
8563,Week 3,Elicit - Week 1-2 | Week 2 : Project Planning ...,KA03_112,NaN
8564,Week 4,Engage - Week 3-5 | Week 4: Review and Experim...,KA03_112,NaN
8565,Week 3,Engage - Week 3-5 | Week 3: Review and Experim...,KA03_112,NaN
8566,Week 4,Engage - Week 3-5 | Week 4: Review and Experim...,KA03_112,NaN


In [153]:
# Create a pivot table for weekly data
pj_pivot = pj_cleaned.pivot_table(index='Mentee ID', columns='Select Week', aggfunc='first')

In [154]:
# Creating new column names by combining the first and second elements
pj_pivot.columns = [f'{col[0]} - {col[1]}' for col in pj_pivot.columns]


In [155]:
# Index reseting 
pj_pivot.reset_index(inplace=True)

In [156]:
# Drop columns that start with 'Attended Sessions'
pj_pivot = pj_pivot.loc[:, ~pj_pivot.columns.str.startswith('Attended Sessions')]

In [157]:
# Making multiple copy of dataframe
aw = pt.copy()
an = pt.copy()
pp = pt.copy()
qa= pt.copy()

In [158]:
# Function to clean and process data
def process_data(df, prefix):
    # Get the columns to keep
    columns_to_keep = [col for col in df.columns if col.startswith("Select Week") or col.startswith(prefix)]
    # Drop the columns not in columns_to_keep
    df = df[columns_to_keep].copy()
    # Extract the week number from the 'Select Week' column
    df['Select Week'] = df['Select Week'].str.split(':').str[0]
    # Remove comment before column names
    df.columns = df.columns.str.replace(prefix, '')
    # Remove extra spaces and unwanted signs from column names
    df.columns = df.columns.str.strip().str.replace('[', '').str.replace(']', '')
    # Rename columns using a lambda function
    df = df.rename(columns=lambda x: '_'.join(x.split('_')[:2]) if '_' in x else x)
    # Melt the dataframe
    df_filter = pd.melt(df, id_vars=['Select Week'], var_name='Mentee ID', value_name=prefix.split(': ')[-1])
    # Convert values to numeric
    #df_filter[prefix.split(': ')[-1]] = pd.to_numeric(df_filter[prefix.split(': ')[-1]], errors='coerce').fillna(0).astype(int)
    # Removing null values 
    df_filter = df_filter.dropna(subset=[prefix.split(': ')[-1]])
    #df_filter[prefix.split(': ')[-1]] = pd.to_numeric(df_filter[prefix.split(': ')[-1]], errors='coerce').fillna(0).astype(int)
    # Create a pivot table
    df_pivot = df_filter.pivot_table(index=['Mentee ID'], columns='Select Week', aggfunc='first')
    # Rename columns by combining parts before and after the hyphen
    df_pivot.columns = [f'{col[0]} - {col[1]}' for col in df_pivot.columns]
    # Reset the index
    df_pivot.reset_index(inplace=True)
    return df_pivot


In [159]:
# Define prefixes
prefixes = [
    "Project Parameter: Awareness about the tasks",
    "Project Parameter: Attentiveness",
    "Project Parameter: Participation",
    "Project Parameter: Quality of task submitted"
]



In [160]:

# Process each dataframe
aw_pivot = process_data(aw, prefixes[0])
an_pivot = process_data(an, prefixes[1])
pp_pivot = process_data(pp, prefixes[2])
qa_pivot = process_data(qa, prefixes[3])

In [161]:
# Perform outer joins
merged = aw_pivot
for pivot in [an_pivot, pp_pivot, qa_pivot]:
    merged = pd.merge(merged, pivot, on='Mentee ID', how='outer')


In [162]:
# Merge with gi
Pro_tra = pd.merge(gi, merged, on='Mentee ID', how='outer')


In [163]:
# Merge with pro_tra
Pro_tra = pd.merge(Pro_tra, pj_pivot, on='Mentee ID', how='outer')


In [164]:
# Remove comment before column names
Pro_tra.columns = Pro_tra.columns.str.replace("Awareness about the tasks", 'Awareness')
Pro_tra.columns = Pro_tra.columns.str.replace("Quality of task submitted", 'Quality')

Pro_tra.columns

In [165]:
def overall_rating(df):
    # Identify the week numbers from column names
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    for week in weeks:
        # Define the column names
        awareness_col = f'Awareness - Week {week}'
        attentiveness_col = f'Attentiveness - Week {week}'
        participation_col = f'Participation - Week {week}'
        quality_col = f'Quality - Week {week}'
        ovv_rat_col = f'Overall Rating - Week {week}'
        
        # Check if all necessary columns are present
        if all(col in df.columns for col in [awareness_col, attentiveness_col, participation_col, quality_col]):
            # Convert columns to numeric, coercing errors to NaN
            df[awareness_col] = pd.to_numeric(df[awareness_col], errors='coerce')
            df[attentiveness_col] = pd.to_numeric(df[attentiveness_col], errors='coerce')
            df[participation_col] = pd.to_numeric(df[participation_col], errors='coerce')
            df[quality_col] = pd.to_numeric(df[quality_col], errors='coerce')
            
            # Calculate the overall rating
            df[ovv_rat_col] = df[awareness_col] + df[attentiveness_col] + df[participation_col] + df[quality_col]
    
    return df


In [166]:
# Applying function
Pro_tra = overall_rating(Pro_tra)


In [167]:
def rearrange_columns(df):
    # Extract week numbers from the columns
    weeks = sorted(set(int(col.split('- Week ')[1]) for col in df.columns if 'Week' in col))
    
    # Define the initial part of the arrangement
    arranged_columns = ['Mentee ID','Email ID','Mentee Name','Mobile Number','Group Name']
    
    # Arrange columns for each week dynamically
    for week in weeks:
        arranged_columns.append(f'Project Details - Week {week}')
        arranged_columns.append(f'Awareness - Week {week}')
        arranged_columns.append(f'Attentiveness - Week {week}')
        arranged_columns.append(f'Participation - Week {week}')
        arranged_columns.append(f'Quality - Week {week}')
        arranged_columns.append(f'Overall Rating - Week {week}')
        
    return arranged_columns

In [168]:
rearrange_columns = rearrange_columns(Pro_tra)
# Rearrange the dataframe
Pro_tra = Pro_tra[rearrange_columns]

In [169]:
# Creating function to get Overall Performance columns

def calculate_mean_scores(df, prefixes):
    # Iterate through each prefix
    for prefix in prefixes:
        # Filter columns that start with the given prefix
        cols = [col for col in df.columns if col.startswith(prefix)]
        # Calculate the sum of scores across columns for the current prefix
        df[f"{prefix} Score"] = df[cols].sum(axis=1)  # Calculate sum score

# Usage of function
prefixes = ["Awareness", "Attentiveness", "Participation", "Quality", "Overall"]
calculate_mean_scores(Pro_tra, prefixes)


In [170]:
# Filling null values with 0
#Pro_tra.fillna(0, inplace=True)


In [171]:
Pro_tra

,Mentee ID,Email ID,Mentee Name,Mobile Number,Group Name,Project Details - Week 1,Awareness - Week 1,Attentiveness - Week 1,Participation - Week 1,Quality - Week 1,Overall Rating - Week 1,Project Details - Week 2,Awareness - Week 2,Attentiveness - Week 2,Participation - Week 2,Quality - Week 2,Overall Rating - Week 2,Project Details - Week 3,Awareness - Week 3,Attentiveness - Week 3,Participation - Week 3,Quality - Week 3,Overall Rating - Week 3,Project Details - Week 4,Awareness - Week 4,Attentiveness - Week 4,Participation - Week 4,Quality - Week 4,Overall Rating - Week 4,Project Details - Week 5,Awareness - Week 5,Attentiveness - Week 5,Participation - Week 5,Quality - Week 5,Overall Rating - Week 5,Awareness Score,Attentiveness Score,Participation Score,Quality Score,Overall Score
0,KA03_01,rangarirohinireddy@gmail.com,Rohini Reddy Rangari,6.281661e+09,BIO_01Neha,Elicit - Week 1-2 | Week 1 : Project Initiatio...,1.0,1.0,1.0,1.0,4.0,Elicit - Week 1-2 | Week 2 : Project Planning ...,3.0,4.0,4.0,3.0,14.0,Engage - Week 3-5 | Week 3: Review and Experim...,4.0,3.0,4.0,2.0,13.0,Engage - Week 3-5 | Week 3: Review and Experim...,1.0,2.0,2.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,10.0,11.0,7.0,37.0
1,KA03_02,namrtad82@gmail.com,Namrta Dixit,9.630908e+09,BIO_01Neha,Elicit - Week 1-2 | Week 1 : Project Initiatio...,4.0,4.0,5.0,3.0,16.0,Elicit - Week 1-2 | Week 2 : Project Planning ...,4.0,4.0,4.0,3.0,15.0,Engage - Week 3-5 | Week 3: Review and Experim...,2.0,1.0,2.0,2.0,7.0,Engage - Week 3-5 | Week 3: Review and Experim...,4.0,5.0,5.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,16.0,11.0,55.0
2,KA03_03,amalendus4nair@gmail.com,Amalendu Sasikumar Nair,9.496757e+09,BIO_01Neha,Elicit - Week 1-2 | Week 1 : Project Initiatio...,1.0,1.0,1.0,1.0,4.0,Elicit - Week 1-2 | Week 2 : Project Planning ...,4.0,4.0,4.0,3.0,15.0,Engage - Week 3-5 | Week 3: Review and Experim...,4.0,4.0,5.0,3.0,16.0,Engage - Week 3-5 | Week 3: Review and Experim...,2.0,3.0,3.0,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,12.0,13.0,9.0,45.0
3,KA03_26,anaswara2928@gmail.com,Anaswara H,7.736603e+09,BIO_01Neha,Elicit - Week 1-2 | Week 1 : Project Initiatio...,1.0,1.0,1.0,1.0,4.0,Elicit - Week 1-2 | Week 2 : Project Planning ...,1.0,1.0,1.0,1.0,4.0,Engage - Week 3-5 | Week 3: Review and Experim...,1.0,1.0,2.0,2.0,6.0,Engage - Week 3-5 | Week 3: Review and Experim...,5.0,3.0,3.0,5.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,6.0,7.0,9.0,30.0
4,KA03_05,swathi.deepak@mbty.christuniversity.in,Swathi Nayak,9.606190e+09,BIO_01Neha,Elicit - Week 1-2 | Week 1 : Project Initiatio...,1.0,1.0,1.0,1.0,4.0,Elicit - Week 1-2 | Week 2 : Project Planning ...,1.0,1.0,1.0,1.0,4.0,Engage - Week 3-5 | Week 3: Review and Experim...,1.0,1.0,1.0,2.0,5.0,Engage - Week 3-5 | Week 3: Review and Experim...,1.0,1.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,5.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,KA03_73,sumayyasubair0786@gmail.com,Sumayya Subair,8.590496e+09,PHY_06Shivani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
132,KA03_75,anjalioli96@gmail.com,Anjali Oli,8.057773e+09,PHY_06Shivani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
133,KA03_76,ishanab355@gmail.com,Ishana B,8.129407e+09,PHY_06Shivani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
134,KA03_77,himanigahtori181@gmail.com,Himani Gahtori,8.057881e+09,PHY_06Shivani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [172]:
# We are doing this to get Overall percentage column as we want to count number of week and total week score is 20 so we are multiply with it
# Count the number of columns that start with 'Overall Rating'
# Function to calculate overall percentage
def calculate_overall_percentage(row):
    overall_ratings = row.filter(like='Overall Rating')
    non_null_ratings = overall_ratings.dropna()
    count_non_null = non_null_ratings.count()
    if count_non_null == 0:
        return np.nan
    overall_percentage = (row['Overall Score'] / (count_non_null * 20)) * 100
    return overall_percentage

# Apply the function to each row
Pro_tra['Overall Percentage'] = Pro_tra.apply(calculate_overall_percentage, axis=1)

In [173]:
# Rounding values 
Pro_tra=Pro_tra.round(2)

In [174]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\Accelerator 3.0\output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Project Performance Tracking.csv')
Pro_tra.to_csv(output_path, index=False)
